<a href="https://colab.research.google.com/github/Park-yonghoon/Capstone_Project/blob/master/Chp05_2_%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""# 교차 검증과 그리드 서치"""

"""## 검증 세트"""

import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

"""## 교차 검증"""

from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

import numpy as np

print(np.mean(scores['test_score']))

from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

"""## hyperparameter 튜닝"""

from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

print(gs.best_params_)

print(gs.cv_results_['mean_test_score'])

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth':range(5, 20, 1),
          'min_samples_split':range(2, 100, 10)
         }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)

print(np.max(gs.cv_results_['mean_test_score']))

"""## random search"""

from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

np.unique(rgen.rvs(1000), return_counts=True)
ugen = uniform(0, 1)
ugen.rvs(10)

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth':randint(20, 50),
          'min_samples_split':randint(2, 25),
          'min_samples_leaf': randint(1, 25),
         }

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)

gs.fit(train_input, train_target)

print(gs.best_params_)

print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_

print(dt.score(test_input, test_target))

(4157, 3) (1040, 3)
0.9971133028626413
0.864423076923077
{'fit_time': array([0.0137701 , 0.01120806, 0.01743364, 0.01290083, 0.01379466]), 'score_time': array([0.00295234, 0.0020628 , 0.00159979, 0.0011096 , 0.00109482]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487
0.855300214703487
0.8574181117533719
0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}
{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731
{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86


In [ ]:
### Keywords
## 검증세트, 하이퍼파라미터 튜닝을 위해 모델을 평가할 때, 테스트 세트를 사용하지 않기 위해 훈련 세트에서 다시 떼어 낸 데이터 세트입니다.
## 교차검증, 훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 모든 폴드에 대해 검증 점수를 얻어 평균하는 방법입니다.
## 그리드 서치, 하이퍼파라미터 탐색을 자동화해 주는 도구입니다. 탐색할 매개변수를 나열하면 교차 검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합을 선택합니다. 마지막으로 이 매개변수 조합으로 최종 모델을 훈련합니다.
## 랜덤 서치는 연속된 매개변수 값을 탐색할 때 유용합니다. 탐색할 값을 직접 나열하는 것이 아니고 탐색 값을 샘플링할 수 있는 확률 분포 객체를 전달합니다. 지정된 횟수만큼 샘플링하여 교차 검증을 수행하기 때문에 시스템 자원이 허락하는 만큼 탐색량을 조절할 수 있습니다.

### Main package & function
## scikit-learn
#   cross_validate()는 교차 검증을 수행하는 함수임.
#     첫 번째 매개변수에 교차 검증을 수행할 모델 객체를 전달함.
#     두 번째와 세 번째 매개변수에 특성과 타깃 데이터를 전달함.
#     scoring 매개변수 검증에 사용할 평가 지표를 지정할 수 있음. 기본적으로 분류 모델은 정확도를 의미하는 'accuracy', 회귀 모델은 결정계수를 의미하는 'r2'가 됨.
#     cv 매개변수에 교차 검증 폴드 수나 스플리터 객체를 지정할 수 있음. 기본값은 5입니다. 회귀일 때는 KFold 클래스를 사용하고 분류일 때는 StratifiedKFold 클래스를 사용하여 5-폴드 교차 검증을 수행함.
#     n_jobs 매개변수는 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정함. 기본값은 1로 하나의 코어를 사용함. -1로 지정하면 시스템에 있는 모든 코어를 사용함.
#     return_train_score 매개변수를 true로 지정하면 훈련 세트의 점수도 변환함. 기본값은 false임.
#   GridSearchCV, 교차 검증으로 하이퍼파라미터 탐색을 수행함. 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련함.
#     첫 번째 매개변수로 그리드 서치를 수행할 모델 객체를 전달함. 두 번째 매개변수에는 탐색할 모델의 매개변수와 값을 전달함.
#     scoring, cv, n_jobs, return_train_score 매개변수는 cross_validate() 함수와 동일함.
#   RandomizedSearchCV, 교차 검증으로 랜덤한 하이퍼파라미터 탐색을 수행함. 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련함.
#     첫 번째 매개변수로 그리드 서치를 수행할 모델 객체를 전달함. 두 번째 매개변수에는 탐색할 모델의 매개변수와 확률 분포 객체를 전달함.
#     scoring, cv, n_jobs, return_train_score 매개변수는 cross_validate() 함수와 동일함.